In [1]:
import collections
import re
from d2l import torch as d2l

In [22]:
# 下载并存储 'time_machine' 数据集的 URL 和哈希值
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                               '090b5e7e70c295757f55df93cb0a180b9691891a')  
def read_time_machine():
    with open(d2l.download('time_machine'),'r') as f:
        lines=f.readlines()
    return [re.sub('[^A-Za-z]+',' ',line).strip().lower() for line in lines]
#re.sub()将所有的非字符串转变为空格
#strip去除前后尾的空格
#lower将大写字母变成小写
lines=read_time_machine()
print(len(lines))
print(lines[10])
print(lines)

3221
twinkled and his usually pale face was flushed and animated the
['the time machine by h g wells', '', '', '', '', 'i', '', '', 'the time traveller for so it will be convenient to speak of him', 'was expounding a recondite matter to us his grey eyes shone and', 'twinkled and his usually pale face was flushed and animated the', 'fire burned brightly and the soft radiance of the incandescent', 'lights in the lilies of silver caught the bubbles that flashed and', 'passed in our glasses our chairs being his patents embraced and', 'caressed us rather than submitted to be sat upon and there was that', 'luxurious after dinner atmosphere when thought roams gracefully', 'free of the trammels of precision and he put it to us in this', 'way marking the points with a lean forefinger as we sat and lazily', 'admired his earnestness over this new paradox as we thought it', 'and his fecundity', '', 'you must follow me carefully i shall have to controvert one or two', 'ideas that are almost univers

In [21]:
def tokenize(lines,token='word'):
    if token=='word':
        return [line.split() for line in lines]
    elif token=='char':
        return [list(line) for line in lines]
    else:
        print('error')
tokens=tokenize(lines,token='word')
print(tokens[10])

['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [33]:
#词汇表
class Vocab:
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens==None:
            tokens=[]
        if reserved_tokens==None:
            reserved_tokens=[]
        counter=count_corpus(tokens)
        self.token_freqs=sorted(counter.items(),key=lambda x:x[1],reverse=True)
        self.unk,uniq_tokens=0,['<unk>']+reserved_tokens
        uniq_tokens+=[token for token,freq in self.token_freqs
                     if freq>=min_freq and token not in uniq_tokens]
        self.idx_to_token=[]
        self.token_to_idx=dict()
        for token in uniq_tokens:
            self.idx_to_token.append(token)
            self.token_to_idx[token]=len(self.idx_to_token)-1
    def __len__(self):
        return len(self.idx_to_token)
    def __getitem__(self,tokens):#根据标记获取其对应的索引或索引列表。
        if not isinstance(tokens,(list,tuple)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]
    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return self.idx_to_tokens[indices]
        return [self.idx_to_tokens[index] for index in indices]
def count_corpus(tokens):
    if len(tokens)==0 or isinstance(tokens[0],list):
        tokens=[token for line in tokens for token in line]
    return collections.Counter(tokens)
 # 使用 collections.Counter 统计标记的频率

In [34]:
vocab=Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [18]:
for i in [0,30]:
    print('word:',tokens[i])
    print('indices:',vocab[tokens[i]])

word: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
indices: [1, 19, 50, 40, 2183, 2184, 400]
word: ['know', 'of', 'course', 'that', 'a', 'mathematical', 'line', 'a', 'line', 'of', 'thickness', 'nil']
indices: [151, 4, 313, 9, 5, 1048, 331, 5, 331, 4, 684, 2212]


In [28]:
#将所有的功能打包到一个函数里面
def load_corpus_time_machine(max_tokens=-1):
    lines=read_time_machine()
    tokens=tokenize(lines,'char')
    vocab=Vocab(tokens)
    corpus=[vocab[token] for line in tokens for token in line]
    if max_tokens>0:
        corpus=corpus[:max_tokens]
    return corpus,vocab
corpus,vacab=load_corpus_time_machine()
print(len(corpus))
print(len(vocab))

170580
4580
